In [2]:
pip install tensorflow keras albumentations opencv-python numpy matplotlib --user

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --user albumentations numpy==1.23.5


  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl.metadata (2.3 kB)
INFO: pip is looking at multiple versions of scikit-image to determine which version is compatible with other requirements. This could take a while.
  Using cached scikit_image-0.25.1-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached scikit_image-0.25.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached scikit_image-0.24.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.9 MB 3.7 MB/s eta 0:00:04
   ------------ --------------------------- 3.9/12.9 MB 10.2 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.9 MB 17.4 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.9 MB 17.4 MB/s eta 0:00:01
   -------------------------- --

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
moviepy 2.1.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
keras-segmentation 0.3.0 requires imageio==2.5.0, but you have imageio 2.37.0 which is incompatible.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 1.23.5 which is incompatible.


In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf 
import albumentations as A
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import tensorflow.keras.backend as K


In [46]:
class TeethDataGenerator(Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=8, image_size=(256, 256), augment=True, class_weighting=True, **kwargs):
        super().__init__(**kwargs)
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        self.class_weighting = class_weighting
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()
        
        self.augmentation = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.4),
            A.GaussianBlur(p=0.3),
            A.ElasticTransform(alpha=2, sigma=50, alpha_affine=50, p=0.5),
            A.RandomCrop(height=256, width=256, p=0.7)
        ])

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = [self.image_paths[k] for k in indexes]
        batch_masks = [self.mask_paths[k] for k in indexes]
        images, masks = self.__data_generation(batch_images, batch_masks)
        return np.array(images), np.array(masks)

    def __data_generation(self, batch_images, batch_masks):
        images, masks = [], [] 
        for img_path, mask_path in zip(batch_images, batch_masks):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, self.image_size) / 255.0  # Normalize image

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.image_size) / 255.0  # Normalize mask
            #Normalize images properly (Scale to 0-255 instead of 0-1)
            img = np.clip(img, 0, 255).astype(np.uint8)  # Ensure dtype uint8
            mask = np.clip(mask, 0, 255).astype(np.uint8)
            # Convert mask to binary: Thresholding (values >= 0.5 become 1, else 0)
            mask = np.where(mask >= 0.5, 1, 0).astype(np.uint8)

            # Mask Check: Ensure only 0 & 1 values
            unique_values = np.unique(mask)
            #print(f"Mask unique values for {mask_path}: {unique_values}")  # Debugging output
            if not np.all(np.isin(unique_values, [0, 1])):
                raise ValueError(f"ERROR: Unexpected mask values in {mask_path}: {unique_values}. Expected only {0, 1}.")


            if self.augment:
                augmented = self.augmentation(image=img, mask=mask)
                img, mask = augmented["image"], augmented["mask"]

            images.append(np.expand_dims(img, axis=-1))
            masks.append(np.expand_dims(mask, axis=-1))

        images, masks = np.array(images), np.array(masks)

        if self.class_weighting:
            mask_weights = np.where(masks > 0.5, 2.0, 1.0)  # Give more weight to teeth pixels
            masks = masks * mask_weights

        return images, masks

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [41]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def se_block(x, ratio=16):
    """Squeeze-and-Excitation Block to improve feature importance."""
    channels = x.shape[-1]
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Dense(channels // ratio, activation="relu")(se)
    se = layers.Dense(channels, activation="sigmoid")(se)
    se = layers.Reshape((1, 1, channels))(se)
    return layers.Multiply()([x, se])

def conv_block(x, num_filters, dilation_rate=1):
    """ Convolutional Block with SE Block and Dilated Convolutions"""
    shortcut = x  # Save input for residual connection

    x = layers.Conv2D(num_filters, (3, 3), padding="same", dilation_rate=dilation_rate)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(num_filters, (3, 3), padding="same", dilation_rate=dilation_rate)(x)
    x = layers.BatchNormalization()(x)

    # Add Squeeze-and-Excitation Block
    x = se_block(x)

    # Add Residual Connection
    if shortcut.shape[-1] != num_filters:
        shortcut = layers.Conv2D(num_filters, (1, 1), padding="same")(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation("relu")(x)

    return x

def upsample_block(x, skip, num_filters):
    """ UpSampling Block with Skip Connection """
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unetpp(input_shape=(256, 256, 1), num_classes=1):
    inputs = layers.Input(input_shape)

    # Encoder
    c1 = conv_block(inputs, 32)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 64, dilation_rate=2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 128, dilation_rate=4)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = conv_block(p3, 256, dilation_rate=8)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = conv_block(p4, 512)

    # Decoder (Nested U-Net++)
    u4_1 = upsample_block(c5, c4, 256)
    u3_1 = upsample_block(u4_1, c3, 128)
    u2_1 = upsample_block(u3_1, c2, 64)
    u1_1 = upsample_block(u2_1, c1, 32)

    # Nested Skip Paths
    u3_2 = upsample_block(u4_1, u3_1, 128)
    u2_2 = upsample_block(u3_2, u2_1, 64)
    u1_2 = upsample_block(u2_2, u1_1, 32)

    u2_3 = upsample_block(u3_2, u2_2, 64)
    u1_3 = upsample_block(u2_3, u1_2, 32)

    u1_4 = upsample_block(u2_3, u1_3, 32)

    # Output layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation="sigmoid")(u1_4)

    return Model(inputs, outputs)

# Build and compile the model
model = build_unetpp(input_shape=(256, 256, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

# Print Model Summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)   │ (None, 256, 256, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_12 (Conv2D)            │ (None, 256, 256, 32)      │             320 │ input_layer_15[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256, 256, 32)      │             128 │ conv2d_12[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 256, 256, 32)      │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_13 (Conv2D)            │ (None, 256, 256, 32)      │           9,248 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 256, 256, 32)      │             128 │ conv2d_13[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d      │ (None, 32)                │               0 │ batch_normalization_1[0][… │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 2)                 │              66 │ global_average_pooling2d[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 32)                │              96 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ reshape (Reshape)             │ (None, 1, 1, 32)          │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multiply (Multiply)           │ (None, 256, 256, 32)      │               0 │ batch_normalization_1[0][… │
│                               │                           │                 │ reshape[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_14 (Conv2D)            │ (None, 256, 256, 32)      │              64 │ input_layer_15[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 256, 256, 32)      │               0 │ multiply[0][0],            │
│                               │                           │                 │ conv2d_14[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 256, 256, 32)      │               0 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 32)      │               

 Total params: 9,436,627 (36.00 MB)

 Trainable params: 9,429,331 (35.97 MB)

 Non-trainable params: 7,296 (28.50 KB)

In [50]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true = K.cast(y_true, 'float32')  
    y_pred = K.cast(y_pred, 'float32')
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (K.sum(y_true) + K.sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def weighted_focal_loss(y_true, y_pred, alpha=0.95, gamma=2.0):
    epsilon = K.epsilon()
    y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
    loss = -alpha * (1 - y_pred) ** gamma * y_true * K.log(y_pred) - \
           (1 - alpha) * y_pred ** gamma * (1 - y_true) * K.log(1 - y_pred)
    return K.mean(loss)

def tversky_loss(y_true, y_pred, smooth=1e-6, alpha=0.7, beta=0.3):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    TP = K.sum(y_true_f * y_pred_f)
    FP = K.sum((1 - y_true_f) * y_pred_f)
    FN = K.sum(y_true_f * (1 - y_pred_f))
    tversky_index = (TP + smooth) / (TP + alpha * FP + beta * FN + smooth)
    return 1 - tversky_index

def combined_loss(y_true, y_pred):
    return 0.4 * dice_loss(y_true, y_pred) + 0.3 * weighted_focal_loss(y_true, y_pred) + 0.3 * tversky_loss(y_true, y_pred)


In [51]:
# Define Dataset Paths
dataset_path = r"C:\Users\FAST\Desktop\FYP_work-Dental\FYP_work\DecayDataSrc\Teeth_Dataset\augmented"
image_dir = os.path.join(dataset_path, "Images1")
mask_dir = os.path.join(dataset_path, "Masks1")

# Get Sorted Image and Mask Paths
image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.png')])

# Ensure Correct Image-Mask Pairing
assert len(image_paths) == len(mask_paths), "Mismatch in number of images and masks!"

# Split Dataset: 80% Train, 20% Validation
split = int(0.8 * len(image_paths))
train_images, val_images = image_paths[:split], image_paths[split:]
train_masks, val_masks = mask_paths[:split], mask_paths[split:]

print(f"Training Samples: {len(train_images)}, Validation Samples: {len(val_images)}")

Training Samples: 464, Validation Samples: 116


In [52]:
# initialize data generators
train_gen = TeethDataGenerator(train_images, train_masks, batch_size=8, augment=True)
val_gen = TeethDataGenerator(val_images, val_masks, batch_size=8, augment=False)

In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
model = build_unetpp(input_shape=(256, 256, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=combined_loss, 
              metrics=[dice_coefficient])
checkpoint_path = "unetpp_best_model_4.keras"
callbacks = [
    # Save the best model based on validation dice coefficient
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_dice_coefficient', 
                    save_best_only=True, mode='max', verbose=1),

    # Reduce Learning Rate if validation loss doesn't improve
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1),

    # Stop training early if no improvement in validation loss
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=80, callbacks=callbacks)

# Save Final Model
model.save("unetpp_final_trained_4.h5")


Epoch 1/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - dice_coefficient: 0.6760 - loss: 0.2618
Epoch 1: val_dice_coefficient improved from -inf to 0.69476, saving model to unetpp_best_model_4.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 430s 7s/step - dice_coefficient: 0.6774 - loss: 0.2609 - val_dice_coefficient: 0.6948 - val_loss: 0.2265 - learning_rate: 1.0000e-04
Epoch 2/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - dice_coefficient: 0.8567 - loss: 0.1373
Epoch 2: val_dice_coefficient improved from 0.69476 to 0.83380, saving model to unetpp_best_model_4.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 404s 7s/step - dice_coefficient: 0.8566 - loss: 0.1373 - val_dice_coefficient: 0.8338 - val_loss: 0.1246 - learning_rate: 1.0000e-04
Epoch 3/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - dice_coefficient: 0.8515 - loss: 0.1384
Epoch 3: val_dice_coefficient improved from 0.83380 to 0.89468, saving model to unetpp_best_model_4.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 405s 7s/step - dice_coefficient: 0.8516 - loss: 0.1383 - val_dice

In [54]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

# ✅ Load the best saved model after early stopping
model = load_model("unetpp_best_model_4.keras", custom_objects={
    "dice_coefficient": dice_coefficient,
    "dice_loss": dice_loss,
    "combined_loss": combined_loss
})
print("✅ Model loaded successfully!")

# ✅ Function to preprocess images
def preprocess_image(image_path, target_size=(256, 256)):
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# ✅ Function to preprocess masks (convert to binary labels)
def preprocess_mask(mask_path, target_size=(256, 256)):
    mask = tf.keras.preprocessing.image.load_img(mask_path, color_mode="grayscale", target_size=target_size)
    mask = tf.keras.preprocessing.image.img_to_array(mask) / 255.0  # Normalize
    mask = (mask > 0.5).astype(np.uint8)  # Convert to binary mask (0 or 1)
    return mask.flatten()  # Flatten mask to 1D array

# ✅ Load validation images and masks
X_val = np.vstack([preprocess_image(img) for img in val_images])  # Stack images into array
y_val = np.concatenate([preprocess_mask(mask) for mask in val_masks])  # Flatten masks

print("✅ Validation data prepared successfully!")

# ✅ Get model predictions
y_pred = model.predict(X_val, batch_size=8)
y_pred = (y_pred > 0.5).astype(np.uint8)  # Convert probabilities to binary (0 or 1)
y_pred = y_pred.flatten()  # Flatten predictions to match ground truth

print("✅ Predictions generated!")

# ✅ Print classification report
print("\n📊 Classification Report:")
print(classification_report(y_val, y_pred, target_names=["Background", "Teeth"]))


✅ Model loaded successfully!
✅ Validation data prepared successfully!
15/15 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step
✅ Predictions generated!

📊 Classification Report:
              precision    recall  f1-score   support

  Background       0.97      0.83      0.89   6010399
       Teeth       0.58      0.89      0.71   1591777

    accuracy                           0.84   7602176
   macro avg       0.78      0.86      0.80   7602176
weighted avg       0.89      0.84      0.85   7602176

